In [1]:
import pandas as pd
from pathlib import Path
from darts import TimeSeries


In [2]:
DATA_ROOT = Path('../../data/')
DS_PATH = DATA_ROOT / 'raw' / 'Количество исследований по неделям (для реализации).csv'
RES_DS_PATH = DATA_ROOT / 'processed' / 'dataset.parquet'

In [3]:
df_raw_ds = pd.read_csv(DS_PATH)
df_raw_ds

,Год,Номер недели,Денситометр,КТ,КТ с КУ 1 зона,КТ с КУ 2 и более зон,ММГ,МРТ,МРТ с КУ 1 зона,МРТ с КУ 2 и более зон,РГ,Флюорограф
0,2021,52,NaN,5895,NaN,NaN,22,8,NaN,NaN,2459,NaN
1,2022,1,17.0,6146,43.0,100.0,483,415,169.0,2.0,12450,392.0
2,2022,2,1026.0,10868,424.0,451.0,9567,2156,669.0,9.0,48904,22626.0
3,2022,3,910.0,12266,430.0,490.0,8791,2162,710.0,14.0,47364,20496.0
4,2022,4,679.0,12793,336.0,471.0,7465,2066,667.0,7.0,40234,15227.0
...,...,...,...,...,...,...,...,...,...,...,...,...
104,2023,52,1294.0,3313,491.0,476.0,14856,1327,715.0,11.0,81751,5979.0
105,2024,1,84.0,950,70.0,73.0,1185,544,131.0,1.0,16148,241.0
106,2024,2,1427.0,3317,431.0,369.0,13964,1461,593.0,12.0,80644,5394.0
107,2024,3,1816.0,3939,563.0,518.0,17769,1712,809.0,17.0,98705,6580.0


In [4]:
# drop 1-st row and fill empty with 0
df_ds = df_raw_ds.loc[1:].fillna(1)
# make timestamp
df_ds['ds'] = pd.to_datetime(df_ds['Год'].astype('str') + '-' + df_ds['Номер недели'].astype('str') + '-1', format="%Y-%W-%w")
# drop columns
df_ds = df_ds.drop(['Год', 'Номер недели'], axis=1)
df_ds

,Денситометр,КТ,КТ с КУ 1 зона,КТ с КУ 2 и более зон,ММГ,МРТ,МРТ с КУ 1 зона,МРТ с КУ 2 и более зон,РГ,Флюорограф,ds
1,17.0,6146,43.0,100.0,483,415,169.0,2.0,12450,392.0,2022-01-03
2,1026.0,10868,424.0,451.0,9567,2156,669.0,9.0,48904,22626.0,2022-01-10
3,910.0,12266,430.0,490.0,8791,2162,710.0,14.0,47364,20496.0,2022-01-17
4,679.0,12793,336.0,471.0,7465,2066,667.0,7.0,40234,15227.0,2022-01-24
5,571.0,13235,302.0,446.0,6124,1900,609.0,6.0,36502,12586.0,2022-01-31
...,...,...,...,...,...,...,...,...,...,...,...
104,1294.0,3313,491.0,476.0,14856,1327,715.0,11.0,81751,5979.0,2023-12-25
105,84.0,950,70.0,73.0,1185,544,131.0,1.0,16148,241.0,2024-01-01
106,1427.0,3317,431.0,369.0,13964,1461,593.0,12.0,80644,5394.0,2024-01-08
107,1816.0,3939,563.0,518.0,17769,1712,809.0,17.0,98705,6580.0,2024-01-15


In [5]:
# make features

# make lag col
def make_lag_column(df, lag_col, feat_col, lag):
    res = df[lag_col].shift(lag).bfill()
    res.name = f"feat_{feat_col}_lag-{lag}_{lag_col}"
    return res

# feature lags
feat_lags = [
    ('КТ', 'КТ с КУ 1 зона', 4),
    ('КТ с КУ 1 зона', 'КТ', 1),
    ('КТ с КУ 2 и более зон', 'МРТ', 6),
    ('МРТ', 'МРТ с КУ 1 зона', 1),
    ('МРТ с КУ 1 зона', 'МРТ', 1),
    ('ММГ', 'КТ с КУ 1 зона', 1)
]
df_res_ds = pd.concat([df_ds]+[make_lag_column(df_ds, *args) for args in feat_lags], axis=1)
df_res_ds

,Денситометр,КТ,КТ с КУ 1 зона,КТ с КУ 2 и более зон,ММГ,МРТ,МРТ с КУ 1 зона,МРТ с КУ 2 и более зон,РГ,Флюорограф,ds,feat_КТ с КУ 1 зона_lag-4_КТ,feat_КТ_lag-1_КТ с КУ 1 зона,feat_МРТ_lag-6_КТ с КУ 2 и более зон,feat_МРТ с КУ 1 зона_lag-1_МРТ,feat_МРТ_lag-1_МРТ с КУ 1 зона,feat_КТ с КУ 1 зона_lag-1_ММГ
1,17.0,6146,43.0,100.0,483,415,169.0,2.0,12450,392.0,2022-01-03,6146.0,43.0,100.0,415.0,169.0,483.0
2,1026.0,10868,424.0,451.0,9567,2156,669.0,9.0,48904,22626.0,2022-01-10,6146.0,43.0,100.0,415.0,169.0,483.0
3,910.0,12266,430.0,490.0,8791,2162,710.0,14.0,47364,20496.0,2022-01-17,6146.0,424.0,100.0,2156.0,669.0,9567.0
4,679.0,12793,336.0,471.0,7465,2066,667.0,7.0,40234,15227.0,2022-01-24,6146.0,430.0,100.0,2162.0,710.0,8791.0
5,571.0,13235,302.0,446.0,6124,1900,609.0,6.0,36502,12586.0,2022-01-31,6146.0,336.0,100.0,2066.0,667.0,7465.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,1294.0,3313,491.0,476.0,14856,1327,715.0,11.0,81751,5979.0,2023-12-25,4473.0,473.0,784.0,1681.0,863.0,16615.0
105,84.0,950,70.0,73.0,1185,544,131.0,1.0,16148,241.0,2024-01-01,4293.0,491.0,800.0,1327.0,715.0,14856.0
106,1427.0,3317,431.0,369.0,13964,1461,593.0,12.0,80644,5394.0,2024-01-08,4764.0,70.0,743.0,544.0,131.0,1185.0
107,1816.0,3939,563.0,518.0,17769,1712,809.0,17.0,98705,6580.0,2024-01-15,4087.0,431.0,782.0,1461.0,593.0,13964.0


In [6]:
# save parqet
df_res_ds.to_parquet(RES_DS_PATH)